In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cassandra/sample_submissions.csv
/kaggle/input/cassandra/train.csv
/kaggle/input/cassandra/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/cassandra/train.csv')
df['Weight'] = pd.to_numeric(df['Weight'],errors='coerce')

In [3]:
df.head(10)

,id,uid,Location,Gender,Age,Height,Weight,Hash,FHO,CHCF,...,NCP,CAEC,SMOKE,Water,CBC,FAF,TI,CA,Mode,Obesity_Level
0,14633,Q68lpI2C3l8tw2QW,SunderNagar,Male,23.000000,1.700000,83.000000,##########,1.0,0,...,3.0,Sometimes,0,3.000000,0,3.000000,1.000000,0,Walking,Overweight_Level_II
1,19443,eUsHPeGtQ1HwdF2p,Punjabi Bagh,Female,23.000000,1.600000,84.000000,##########,1.0,1,...,3.0,Sometimes,0,3.000000,0,3.000000,0.000000,0,Public_Transportation,Obesity_Type_I
2,10216,swftYdUOhKVvaKI8,VikasPuri,Female,28.825223,1.549748,65.220249,##########,1.0,0,...,3.0,Sometimes,0,1.984323,0,1.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
3,2016,U3xIAw1dohYe23cM,Vasant Vihar,Male,25.341399,1.787195,115.428276,##########,1.0,1,...,3.0,Sometimes,0,2.535127,0,0.849811,1.899330,Sometimes,Public_Transportation,Obesity_Type_II
4,9674,MrZ5Tzad8D250EPc,Vasant Vihar,Male,22.000000,1.750000,58.000000,##########,0.0,1,...,4.0,Sometimes,0,2.000000,0,2.000000,0.000000,0,Public_Transportation,Insufficient_Weight
5,19276,T5jAG88dk9Wmx3EB,SunderNagar,Female,39.214514,1.584782,80.000000,##########,1.0,1,...,3.0,Sometimes,0,1.459511,0,0.000000,0.000000,0,Automobile,Obesity_Type_I
6,14752,uE9kQveMq6GxLgZ5,Hauz Khas,Female,25.930376,1.715820,112.956110,##########,1.0,1,...,3.0,Sometimes,0,2.738485,0,0.037202,0.153559,Sometimes,Public_Transportation,Obesity_Type_III
7,8359,1uKhf5QkamG53shw,Vasant Vihar,Male,29.000000,1.630000,66.500000,##########,0.0,0,...,3.0,Frequently,0,2.000000,0,2.000000,1.000000,Sometimes,Public_Transportation,Overweight_Level_I
8,15537,oJPwrCusQjWvfaMl,Punjabi Bagh,Female,18.000000,1.620000,52.000000,##########,0.0,1,...,3.0,Sometimes,0,1.000000,0,0.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
9,4604,BsdNFNfnTZCZX7qt,Punjabi Bagh,Female,25.722004,1.684768,104.774144,##########,1.0,1,...,3.0,Sometimes,0,1.674061,0,0.167943,0.546137,Sometimes,Public_Transportation,Obesity_Type_III


In [4]:
df.shape

(12454, 21)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12454 entries, 0 to 12453
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             12454 non-null  int64  
 1   uid            12454 non-null  object 
 2   Location       12454 non-null  object 
 3   Gender         12454 non-null  object 
 4   Age            12454 non-null  float64
 5   Height         12422 non-null  float64
 6   Weight         12424 non-null  float64
 7   Hash           12441 non-null  object 
 8   FHO            12442 non-null  float64
 9   CHCF           12454 non-null  int64  
 10  CV             12454 non-null  float64
 11  NCP            12454 non-null  float64
 12  CAEC           12440 non-null  object 
 13  SMOKE          12454 non-null  int64  
 14  Water          12454 non-null  float64
 15  CBC            12454 non-null  int64  
 16  FAF            12454 non-null  float64
 17  TI             12439 non-null  float64
 18  CA    

In [6]:
df.isnull().sum()

id                0
uid               0
Location          0
Gender            0
Age               0
Height           32
Weight           30
Hash             13
FHO              12
CHCF              0
CV                0
NCP               0
CAEC             14
SMOKE             0
Water             0
CBC               0
FAF               0
TI               15
CA                0
Mode              0
Obesity_Level     0
dtype: int64

In [7]:
df  = df.drop(columns=['id','uid','Hash','SMOKE','Location'])

In [8]:
df['Height'] = df['Height'].fillna((df['Height'].mean()))
df['Weight'] = df['Weight'].fillna((df['Weight'].mean()))
df['FHO'] = df['FHO'].fillna(df['FHO'].mode()[0])
df['CAEC'] = df['CAEC'].fillna(df['CAEC'].mode()[0])
df['TI'] = df['TI'].fillna(df['TI'].mode()[0])

In [9]:
df.isnull().sum()

Gender           0
Age              0
Height           0
Weight           0
FHO              0
CHCF             0
CV               0
NCP              0
CAEC             0
Water            0
CBC              0
FAF              0
TI               0
CA               0
Mode             0
Obesity_Level    0
dtype: int64

In [10]:
# Location_change ={'SunderNagar':0,'Punjabi Bagh':1,'VikasPuri':2, 'Vasant Vihar':3, 'Hauz Khas':4}
Gender_change ={'Male':0, 'Female':1}
CAEC_change ={'Sometimes':0, 'Frequently':1, 'Always':2, '0':4}
CA_change ={'0':0, 'Sometimes':1, 'Frequently':2}
Mode_change ={'Walking':0, 'Public_Transportation':1, 'Automobile':2, 'Motorbike':3, 'Bike':4, ' ':5}
Obesity_Level_change = {'Overweight_Level_II':0, 'Obesity_Type_I':1, 'Obesity_Type_II':2,'Insufficient_Weight':3, 'Obesity_Type_III':4, 'Overweight_Level_I':5, 'Normal_Weight':6}

In [11]:
# df['Location'] = df['Location'].map(Location_change)
df['Gender'] = df['Gender'].map(Gender_change)
df['CAEC'] = df['CAEC'].map(CAEC_change)
df['CA'] = df['CA'].map(CA_change)
df['Mode'] = df['Mode'].map(Mode_change)
df['Obesity_Level'] = df['Obesity_Level'].map(Obesity_Level_change)

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
Y = df['Obesity_Level']
X = df.drop(columns=['Obesity_Level'])

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=5)

In [15]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(9963, 15) (2491, 15) (9963,) (2491,)


In [16]:
scaler = StandardScaler()

In [17]:
X_train_scaler = scaler.fit_transform(X_train)

In [18]:
X_test_scaler = scaler.transform(X_test)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define hyperparameter search space
param_grid = {
    'learning_rate': [0.05, 0.1, 0.3],
    'max_depth': [3, 5, 8],
    'min_child_weight': [1, 3, 5],
    # Add more hyperparameters as needed
}

# Create the XGBoost model
xgb_model = xgb.XGBClassifier()

# Perform GridSearchCV with early stopping
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5)

# Train the model with GridSearchCV
grid_search.fit(X_train_scaler, Y_train)

# Get the best model with tuned hyperparameters
best_model = grid_search.best_estimator_

# Make predictions with the best model
predictions = best_model.predict(X_test_scaler)

# Evaluate the model
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy with Tuned Hyperparameters:", accuracy)

Accuracy with Tuned Hyperparameters: 0.9088719389803291


In [22]:
# # instantiate the classifier with n_estimators = 100

# rfc_100 = RandomForestClassifier(n_estimators=100, random_state=5)



# # fit the model to the training set

# rfc_100.fit(X_train_scaler, Y_train)



# # Predict on the test set results
# y_pred_100_train = rfc_100.predict(X_train_scaler)
# y_pred_100_test = rfc_100.predict(X_test_scaler)



# # Check accuracy score 
# print('Model accuracy score with 1000 decision-trees with train data : {0:0.4f}'. format(accuracy_score(Y_train, y_pred_100_train)))

# print('Model accuracy score with 1000 decision-trees with test data : {0:0.4f}'. format(accuracy_score(Y_test, y_pred_100_test)))


In [23]:
# Y1 = df['Obesity_Level']
# X1 = df.drop(columns=['Obesity_Level'])
# X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=0.1,random_state=42)
# print(X_train1.shape,X_test1.shape,Y_train1.shape,Y_test1.shape)
# X_train_scaler1 = scaler.fit_transform(X_train1)
# X_test_scaler1 = scaler.transform(X_test1)
# # instantiate the classifier with n_estimators = 100

# rfc_1001 = RandomForestClassifier(n_estimators=100, random_state=5)



# # fit the model to the training set

# rfc_1001.fit(X_train_scaler1, Y_train1)



# # Predict on the test set results
# y_pred_100_train1 = rfc_1001.predict(X_train_scaler1)
# y_pred_100_test1 = rfc_1001.predict(X_test_scaler1)



# # Check accuracy score 
# print('Model accuracy score with 1000 decision-trees with train data : {0:0.4f}'. format(accuracy_score(Y_train1, y_pred_100_train1)))

# print('Model accuracy score with 1000 decision-trees with test data : {0:0.4f}'. format(accuracy_score(Y_test1, y_pred_100_test1)))


In [24]:
df_test = pd.read_csv('/kaggle/input/cassandra/test.csv')
df_test['Weight'] = pd.to_numeric(df_test['Weight'],errors='coerce')

In [25]:
df_test.head(10)

,id,uid,Location,Gender,Age,Height,Weight,Hash,FHO,CHCF,CV,NCP,CAEC,SMOKE,Water,CBC,FAF,TI,CA,Mode
0,10317,7w53s2NgM2arZAvM,Punjabi Bagh,Female,25.951979,1.669039,104.954291,##########,1.0,1,3.000000,3.000000,Sometimes,0,2.092326,0,0.043101,0.540812,Sometimes,Public_Transportation
1,4074,kTLn00K8evgYXWxR,Hauz Khas,Female,40.993179,1.567973,81.022119,##########,1.0,1,2.921576,3.000000,Sometimes,0,1.005727,0,1.034031,0.000000,0,Automobile
2,9060,81HX39uej0BR1xIl,Vasant Vihar,Male,21.000000,1.740000,76.000000,##########,0.0,1,3.000000,3.000000,Sometimes,0,3.000000,0,1.000000,0.000000,Sometimes,Public_Transportation
3,11286,DA9eqs9fs7wJUIet,Hauz Khas,Male,33.000000,1.680000,83.000000,##########,1.0,1,2.000000,1.000000,Sometimes,0,2.000000,0,1.000000,0.000000,Sometimes,Automobile
4,8254,SJ1Z7TBmojAPlJXd,SunderNagar,Female,35.194089,1.673482,73.193589,##########,1.0,0,3.000000,3.000000,Sometimes,0,1.572371,0,1.806740,0.000000,Sometimes,Automobile
5,13071,t6XuQnH7NtkbUt4d,Vasant Vihar,Female,23.000000,1.600000,93.000000,##########,1.0,1,2.000000,3.000000,Sometimes,0,3.000000,0,1.000000,0.000000,0,Public_Transportation
6,11417,oCCVZ0XR5walkNVg,Vasant Vihar,Male,18.000000,1.760000,55.000000,##########,0.0,1,2.000000,4.000000,Sometimes,0,2.000000,0,2.000000,1.000000,Sometimes,Public_Transportation
7,6315,kCsK3ua8M4aPR7on,VikasPuri,Male,33.000000,1.700000,84.000000,##########,1.0,0,3.000000,3.000000,Sometimes,0,3.000000,0,0.000000,0.000000,Sometimes,Automobile
8,14351,xjUeAYsYEzPI3EXQ,VikasPuri,Female,22.978655,1.663632,133.952675,##########,1.0,1,3.000000,3.000000,Sometimes,0,2.408442,0,1.419473,0.843283,Sometimes,Public_Transportation
9,18914,qDCDCXw0mj164rEn,Hauz Khas,Female,23.455324,1.557943,79.697278,##########,1.0,0,1.766849,2.865657,Sometimes,0,1.000000,0,0.990642,1.584830,0,Public_Transportation


In [26]:
df_test = df_test.drop(columns=['id','uid','Hash','SMOKE','Location'])

In [27]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8304 entries, 0 to 8303
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gender  8304 non-null   object 
 1   Age     8304 non-null   float64
 2   Height  8288 non-null   float64
 3   Weight  8279 non-null   float64
 4   FHO     8296 non-null   float64
 5   CHCF    8304 non-null   int64  
 6   CV      8304 non-null   float64
 7   NCP     8304 non-null   float64
 8   CAEC    8298 non-null   object 
 9   Water   8304 non-null   float64
 10  CBC     8304 non-null   int64  
 11  FAF     8304 non-null   float64
 12  TI      8299 non-null   float64
 13  CA      8304 non-null   object 
 14  Mode    8304 non-null   object 
dtypes: float64(9), int64(2), object(4)
memory usage: 973.2+ KB


In [28]:
df_test.isnull().sum()

Gender     0
Age        0
Height    16
Weight    25
FHO        8
CHCF       0
CV         0
NCP        0
CAEC       6
Water      0
CBC        0
FAF        0
TI         5
CA         0
Mode       0
dtype: int64

In [29]:
df_test['Height'] = df_test['Height'].fillna((df_test['Height'].mean()))
df_test['Weight'] = df_test['Weight'].fillna((df_test['Weight'].mean()))
df_test['FHO'] = df_test['FHO'].fillna(df_test['FHO'].mode()[0])
df_test['CAEC'] = df_test['CAEC'].fillna(df_test['CAEC'].mode()[0])
df_test['TI'] = df_test['TI'].fillna(df_test['TI'].mode()[0])

In [30]:
# Location_change_test ={'SunderNagar':0,'Punjabi Bagh':1,'VikasPuri':2, 'Vasant Vihar':3, 'Hauz Khas':4}
Gender_change_test ={'Male':0, 'Female':1}
CAEC_change_test ={'Sometimes':0, 'Frequently':1, 'Always':2, '0':4}
CA_change_test ={'0':0, 'Sometimes':1, 'Frequently':2}
Mode_change_test ={'Walking':0, 'Public_Transportation':1, 'Automobile':2, 'Motorbike':3, 'Bike':4, ' ':5}
# Obesity_Level_change_test = {'Overweight_Level_II':0, 'Obesity_Type_I':1, 'Obesity_Type_II':2,'Insufficient_Weight':3, 'Obesity_Type_III':4, 'Overweight_Level_I':5, 'Normal_Weight':6}

In [31]:
# df_test['Location'] = df_test['Location'].map(Location_change_test)
df_test['Gender'] = df_test['Gender'].map(Gender_change_test)
df_test['CAEC'] = df_test['CAEC'].map(CAEC_change_test)
df_test['CA'] = df_test['CA'].map(CA_change_test)
df_test['Mode'] = df_test['Mode'].map(Mode_change_test)


In [32]:
# test = scaler.transform(df_test)
test = scaler.transform(df_test)

In [33]:
test_pred = best_model.predict(test)

In [34]:
tt = pd.read_csv('/kaggle/input/cassandra/test.csv')

In [35]:
result=pd.DataFrame(tt['id'])
result['Obesity_Level']=test_pred

In [36]:
Obesity_Level_change_rev = dict(map(reversed, Obesity_Level_change.items()))

In [37]:
print(Obesity_Level_change_rev)

{0: 'Overweight_Level_II', 1: 'Obesity_Type_I', 2: 'Obesity_Type_II', 3: 'Insufficient_Weight', 4: 'Obesity_Type_III', 5: 'Overweight_Level_I', 6: 'Normal_Weight'}


In [38]:
result['Obesity_Level'] = result['Obesity_Level'].map(Obesity_Level_change_rev)

In [39]:
result.rename(columns = {'id':'ID'}, inplace = True) 

In [40]:
result.to_csv('result2',index=False,header=1)